In [12]:
!pip cache purge


Files removed: 1000 (587.7 MB)


In [9]:
!pip uninstall -y openai httpx

Found existing installation: openai 1.42.0
Uninstalling openai-1.42.0:
  Successfully uninstalled openai-1.42.0
Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1


In [1]:
!pip install "openai==1.35.13" "httpx==0.27.0"


  Attempting uninstall: openai
    Found existing installation: openai 1.42.0
    Uninstalling openai-1.42.0:
      Successfully uninstalled openai-1.42.0



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# ===============================================================
# PROMPT UNDERSTANDING MODULE (Step 1 of Media Fetching Framework)
# ===============================================================

!pip install openai==1.42.0 pandas matplotlib seaborn

import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from openai import OpenAI
import os
from dotenv import load_dotenv
# =========================

# 🔧 SETUP
# =========================

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))# Helper function for OpenAI call (new SDK syntax)
def ask_llm(prompt, model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a reasoning engine that analyzes creative prompts."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=700
    )
    return completion.choices[0].message.content

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def ask_llm(prompt, model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a reasoning engine that analyzes creative prompts."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=700
    )
    return completion.choices[0].message.content



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# ===============================================================
# PROMPT UNDERSTANDING MODULE (Step 1 of Media Fetching Framework)
# ===============================================================

import openai
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# =========================
# 🧠 STEP 1: Intent Extraction
# =========================

user_prompt = input("Enter your media generation prompt: ")

intent_prompt = f"""
You are an AI system analyzing a creative prompt for intent and structure.

Prompt: "{user_prompt}"

Perform Intent Extraction with these substeps:
1. **Modality** — photo, video, animation, artwork, etc.
2. **Domain** — natural, artistic, conceptual, surreal, etc.
3. **Primary Subject** — the main focus entity.
4. **Context / Scene** — environment or setting.
5. **Style Adjectives** — mood or tone modifiers (e.g. cinematic, slow-motion, dreamy).

Return a JSON object exactly in this format:
{{
  "modality": "",
  "domain": "",
  "primary_subject": "",
  "context_scene": "",
  "style_adjectives": []
}}
"""

intent_json = ask_llm(intent_prompt)
print("🔍 Intent Extraction Result:\n", intent_json)

try:
    intent_data = json.loads(intent_json)
except json.JSONDecodeError:
    print("⚠️ Error decoding JSON — please check model output.")
    intent_data = {}


# =========================
# 🎨 STEP 2: Realism & Abstractness Scoring
# =========================

score_prompt = f"""
You are a cognitive model that evaluates the realism and abstractness of a visual prompt.

Use the following extracted intent:
{json.dumps(intent_data, indent=2)}

Perform analysis based on these three information sources:
1. Subject–Context Relationship: how plausible is the subject existing in this context?
2. Modifiers: do adjectives or style words (like 'glowing', 'ethereal', 'cinematic') add creative abstraction?
3. Domain Knowledge: using world understanding, judge how realistic this concept would be.

From this reasoning, generate TWO quantitative scores between 0.0 and 1.0:
- realism_score: higher means physically and contextually plausible.
- abstractness_score: higher means more imaginative, conceptual, or surreal.

Also provide a short rationale describing your reasoning.

Return ONLY valid JSON in the format:
{{
  "realism_score": 0.0,
  "abstractness_score": 0.0,
  "rationale": "concise explanation summarizing how subject-context, modifiers, and domain knowledge influenced the scores."
}}
"""

scores_json = ask_llm(score_prompt)
print("\n🎯 Realism & Abstractness Scoring:\n", scores_json)

try:
    score_data = json.loads(scores_json)
except json.JSONDecodeError:
    print("⚠️ Could not parse scores JSON.")
    score_data = {}

# =========================
# 🌍 STEP 3: Feasibility Judgement
# =========================

feasibility_prompt = f"""
Now, using your world knowledge, make an overall feasibility judgement.

Given:
Intent: {json.dumps(intent_data, indent=2)}
Scores: {json.dumps(score_data, indent=2)}

Evaluate:
- Is this scenario physically plausible?
- Would an image/video generator find real reference material for this?
- How realistic vs. creative is this concept overall?

Return a JSON with:
{{
  "feasibility_label": "feasible / partially_feasible / fantasy",
  "realism_overall_score": 0.0,
  "creative_potential_score": 0.0,
  "summary": "short explanation"
}}
"""

feasibility_json = ask_llm(feasibility_prompt)
print("\n🌐 Feasibility Judgement:\n", feasibility_json)

try:
    feasibility_data = json.loads(feasibility_json)
except json.JSONDecodeError:
    print("⚠️ Could not parse feasibility JSON.")
    feasibility_data = {}


# =========================
# 🧩 STEP 4: Semantic Map Assembly
# =========================

semantic_map = {
    "user_prompt": user_prompt,
    "intent_extraction": intent_data,
    "realism_scoring": score_data,
    "feasibility_judgement": feasibility_data
}

print("\n🗺️ Semantic Map:\n")
print(json.dumps(semantic_map, indent=2))

# Save for later use
with open("semantic_map.json", "w") as f:
    json.dump(semantic_map, f, indent=2)

# =========================
# 📊 STEP 5: Visualize Scores
# =========================

# Combine numeric scores
numeric_scores = {
    "Subject+Context": score_data.get("subject_context_score", 0),
    "Modifier Abstractness": score_data.get("modifier_abstractness_score", 0),
    "Domain Feasibility": score_data.get("domain_feasibility_score", 0),
    "Overall Realism": feasibility_data.get("realism_overall_score", 0),
    "Creative Potential": feasibility_data.get("creative_potential_score", 0),
}

df = pd.DataFrame(list(numeric_scores.items()), columns=["Dimension", "Score"])

print("\n'Semantic_map.json' saved for next pipeline steps.")


🔍 Intent Extraction Result:
 {
  "modality": "photo",
  "domain": "natural",
  "primary_subject": "cup of coffee",
  "context_scene": "rainy morning window sill",
  "style_adjectives": []
}

🎯 Realism & Abstractness Scoring:
 {
  "realism_score": 0.9,
  "abstractness_score": 0.1,
  "rationale": "The subject of a cup of coffee is highly plausible in the context of a rainy morning on a window sill, as it evokes a common and relatable scene. There are no modifiers present to introduce abstraction, reinforcing the realism of the scenario. Overall, this concept is very realistic and lacks imaginative elements."
}

🌐 Feasibility Judgement:
 {
  "feasibility_label": "feasible",
  "realism_overall_score": 0.9,
  "creative_potential_score": 0.1,
  "summary": "The scenario of a cup of coffee on a rainy morning window sill is highly plausible and relatable, making it feasible for image generation. The high realism score reflects its grounded nature, while the low creative potential indicates a la

A cat sleeping on a sunny window sill with gentle morning light.

A glowing jellyfish flying over a city at night in cinematic slow motion.

A futuristic train made of glass traveling across floating islands.

An astronaut planting flowers on the surface of Mars.

A black-and-white photograph of an old man reading under a streetlamp.

A dragon curled up inside a modern-day subway station.

A realistic portrait of a woman made entirely of blooming roses.

A time-lapse video of clouds forming the shape of running horses.

A candle melting backward, reconstructing itself in reverse motion.

A neon-lit cyberpunk street with rain reflections and flying cars.

A microscopic view of snowflakes forming musical notes.

A realistic dog jumping into a swimming pool during summer.

A painting of an ocean made of glass with sunlight refracting inside.

A video showing trees breathing like living creatures at dusk.

A simple photo of a cup of coffee on a rainy morning window sill.

In [ ]:
# ===============================================================
# DECISION REASONING MODULE (Step 2 of Media Fetching Framework)
# ===============================================================

import json
from openai import OpenAI

# Load your semantic map
with open("semantic_map.json", "r") as f:
    semantic_map = json.load(f)

# ===============================================================
# DECISION REASONING MODULE (Step 2 of Media Fetching Framework)
# ===============================================================

import json
from openai import OpenAI

# Load your semantic map
with open("semantic_map.json", "r") as f:
    semantic_map = json.load(f)

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))    
def ask_llm(prompt, model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an AI decision reasoning module for a media-fetching system."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=800
    )
    return completion.choices[0].message.content


# =========================
# 🧠 STEP 1: Contextual Chain-of-Thought Reasoning
# =========================

decision_prompt = f"""
You are performing *Decision Reasoning* based on the following semantic map:

{json.dumps(semantic_map, indent=2)}

Your task:

1. **Perform contextual reasoning**:
   - Examine the realism_score and abstractness_score.
   - Consider the feasibility_label and creative_potential_score.
   - Think step-by-step about what kind of media retrieval makes sense.

2. **Estimate Cost and Latency**:
   Estimate the computational cost and time latency for both options:
   - Fetch (search or retrieve real media)
   - Generate (create synthetic image/video)
   Classify each as: "low", "medium", or "high" cost and latency.

3. **Decide Fetch vs Generate**:
   Based on your reasoning, choose ONE of:
   - "fetch_from_database_or_web"
   - "generate_with_model"
   - "hybrid_fetch_and_enhance"

4. Return the result as JSON in this format:
{{
  "reasoning_trace": "Your short reasoning chain-of-thought (natural language, concise).",
  "cost_latency_estimate": {{
      "fetch": {{"cost": "", "latency": ""}},
      "generate": {{"cost": "", "latency": ""}}
  }},
  "final_decision": "",
  "confidence": 0.0
}}
"""

decision_json = ask_llm(decision_prompt)
print("\n🧩 Decision Reasoning Result:\n", decision_json)

try:
    decision_data = json.loads(decision_json)
except json.JSONDecodeError:
    print("⚠️ Could not parse decision JSON.")
    decision_data = {}


# =========================
# 📊 STEP 2: Display Decision Summary
# =========================

print("\n🧠 Decision Reasoning Summary:")
print(json.dumps(decision_data, indent=2))

# Merge back into master pipeline
semantic_map["decision_reasoning"] = decision_data

# Save updated map
with open("semantic_map.json", "w") as f:
    json.dump(semantic_map, f, indent=2)

print("\n✅ Updated semantic_map.json saved with decision reasoning results.")

# =========================
# 🧠 STEP 1: Contextual Chain-of-Thought Reasoning
# =========================

decision_prompt = f"""
You are performing *Decision Reasoning* based on the following semantic map:

{json.dumps(semantic_map, indent=2)}

Your task:

1. **Perform contextual reasoning**:
   - Examine the realism_score and abstractness_score.
   - Consider the feasibility_label and creative_potential_score.
   - Think step-by-step about what kind of media retrieval makes sense.

2. **Estimate Cost and Latency**:
   Estimate the computational cost and time latency for both options:
   - Fetch (search or retrieve real media)
   - Generate (create synthetic image/video)
   Classify each as: "low", "medium", or "high" cost and latency.

3. **Decide Fetch vs Generate**:
   Based on your reasoning, choose ONE of:
   - "fetch_from_web"
   - "generate_with_model"
   - "hybrid_fetch_and_enhance"

4. Return the result as JSON in this format:
{{
  "reasoning_trace": "Your short reasoning chain-of-thought (natural language, concise).",
  "cost_latency_estimate": {{
      "fetch": {{"cost": "", "latency": ""}},
      "generate": {{"cost": "", "latency": ""}}
  }},
  "final_decision": "",
  "confidence": 0.0
}}
"""

decision_json = ask_llm(decision_prompt)
print("\n🧩 Decision Reasoning Result:\n", decision_json)

try:
    decision_data = json.loads(decision_json)
except json.JSONDecodeError:
    print("⚠️ Could not parse decision JSON.")
    decision_data = {}


# =========================
# 📊 STEP 2: Display Decision Summary
# =========================

print("\n🧠 Decision Reasoning Summary:")
print(json.dumps(decision_data, indent=2))

# Merge back into master pipeline
semantic_map["decision_reasoning"] = decision_data

# Save updated map
with open("semantic_map.json", "w") as f:
    json.dump(semantic_map, f, indent=2)

print("\n✅ Updated semantic_map.json saved with decision reasoning results.")



🧩 Decision Reasoning Result:
 {
  "reasoning_trace": "The scenario described is highly realistic and relatable, making it feasible to retrieve a real photo of a cup of coffee on a rainy morning window sill. The high realism score (0.9) and low abstractness score (0.1) suggest that this scene is common and easily found in existing media. Given the feasibility label is 'feasible' and the creative potential score is low, it indicates that generating a synthetic image may not add significant value. Therefore, fetching a real image is the most logical choice.",
  "cost_latency_estimate": {
      "fetch": {"cost": "low", "latency": "medium"},
      "generate": {"cost": "medium", "latency": "high"}
  },
  "final_decision": "fetch_from_database_or_web",
  "confidence": 0.95
}

🧠 Decision Reasoning Summary:
{
  "reasoning_trace": "The scenario described is highly realistic and relatable, making it feasible to retrieve a real photo of a cup of coffee on a rainy morning window sill. The high rea

In [ ]:
# scrape_unsplash_and_pexels_selenium.py
# requirements:
# pip install selenium webdriver-manager requests matplotlib pillow
# (This script uses webdriver-manager to auto-download chromedriver)
#
# Note: scraping websites may be restricted by their terms/robots.txt.
# You said the previous Selenium approach worked for you — this script follows
# the same pattern and scrapes both Unsplash and Pexels search pages.
# Use responsibly.

import os
import time
import json
import hashlib
import requests
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# -----------------------------
# Config
# -----------------------------
DEST_DIR = "scrape_assets"
IMAGES_DIR = os.path.join(DEST_DIR, "images")
os.makedirs(IMAGES_DIR, exist_ok=True)
METADATA_PATH = os.path.join(DEST_DIR, "metadata.json")

USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
MIN_PIXELS = 120 * 120  # minimum pixel area to keep an image
HEADLESS = True

# -----------------------------
# WebDriver
# -----------------------------
def create_driver(headless=HEADLESS):
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f"user-agent={USER_AGENT}")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.set_page_load_timeout(60)
    return driver

# -----------------------------
# Utility helpers
# -----------------------------
def choose_best_image_src(src, srcset):
    if srcset:
        try:
            parts = [p.strip() for p in srcset.split(",") if p.strip()]
            last = parts[-1]
            url = last.split(" ")[0]
            return url
        except Exception:
            pass
    return src

def sha256_bytes(b):
    import hashlib
    return hashlib.sha256(b).hexdigest()

def download_bytes(url, timeout=20):
    headers = {"User-Agent": USER_AGENT}
    r = requests.get(url, headers=headers, timeout=timeout, stream=True)
    r.raise_for_status()
    content = r.content
    return content

def inspect_image_bytes(content):
    try:
        im = Image.open(BytesIO(content))
        w, h = im.size
        return w, h
    except Exception:
        return 0, 0

# -----------------------------
# Site-specific scrapers (Selenium + heuristics)
# -----------------------------
def scrape_unsplash_search(driver, query, max_images=20, scroll_pause=1.2):
    """Return list of candidate dicts: {img_url, alt, photographer, page_url, source}"""
    url = f"https://unsplash.com/s/photos/{query.replace(' ', '-')}"
    driver.get(url)
    time.sleep(2)
    # scroll to lazy load
    last_h = driver.execute_script("return document.body.scrollHeight")
    imgs_found = []
    seen = set()
    while len(imgs_found) < max_images:
        imgs = driver.find_elements(By.TAG_NAME, "img")
        for img in imgs:
            try:
                src = img.get_attribute("src") or ""
                srcset = img.get_attribute("srcset") or ""
                alt = img.get_attribute("alt") or ""
                best = choose_best_image_src(src, srcset)
                if not best:
                    continue
                # prefer unsplash-hosted images
                if "images.unsplash.com" not in best and "/photos/" not in best and "photo-" not in best:
                    continue
                dedupe = best.split("?")[0]
                if dedupe in seen:
                    continue
                # attempt to find photographer/page link
                photographer = ""
                page_url = ""
                try:
                    parent_a = img.find_element(By.XPATH, "./ancestor::a[contains(@href,'unsplash.com/photos')][1]")
                    page_url = parent_a.get_attribute("href") or ""
                    # photographer often available in parent figure or siblings; fallback to alt
                except Exception:
                    # fallback: find nearest link
                    try:
                        any_a = img.find_element(By.XPATH, "./ancestor::a[1]")
                        page_url = any_a.get_attribute("href") or ""
                    except Exception:
                        page_url = ""
                # Add candidate
                seen.add(dedupe)
                imgs_found.append({
                    "img_url": best,
                    "alt": alt,
                    "photographer": photographer,
                    "page_url": page_url,
                    "source": "unsplash"
                })
                if len(imgs_found) >= max_images:
                    break
            except Exception:
                continue
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            # small wait and break if still same
            time.sleep(scroll_pause * 1.5)
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break
        last_h = new_h
    return imgs_found[:max_images]

def scrape_pexels_search(driver, query, max_images=20, scroll_pause=1.2):
    """Return list of candidate dicts for Pexels search pages."""
    url = f"https://www.pexels.com/search/{query.replace(' ', '%20')}/"
    driver.get(url)
    time.sleep(2)
    last_h = driver.execute_script("return document.body.scrollHeight")
    imgs_found = []
    seen = set()
    while len(imgs_found) < max_images:
        # Pexels uses many <img> tags; prefer images with 'src' or 'data-src'
        imgs = driver.find_elements(By.TAG_NAME, "img")
        for img in imgs:
            try:
                src = img.get_attribute("src") or img.get_attribute("data-src") or ""
                srcset = img.get_attribute("srcset") or ""
                alt = img.get_attribute("alt") or ""
                best = choose_best_image_src(src, srcset) or src
                if not best:
                    continue
                # skip tiny UI avatars or badges
                if "images.pexels.com" not in best and "pexels" not in best:
                    continue
                dedupe = best.split("?")[0]
                if dedupe in seen:
                    continue
                # find parent link (pin/photo page)
                page_url = ""
                try:
                    parent_a = img.find_element(By.XPATH, "./ancestor::a[contains(@href,'/photo/')][1]")
                    page_url = parent_a.get_attribute("href") or ""
                except Exception:
                    try:
                        any_a = img.find_element(By.XPATH, "./ancestor::a[1]")
                        page_url = any_a.get_attribute("href") or ""
                    except Exception:
                        page_url = ""
                seen.add(dedupe)
                imgs_found.append({
                    "img_url": best,
                    "alt": alt,
                    "photographer": "",
                    "page_url": page_url,
                    "source": "pexels"
                })
                if len(imgs_found) >= max_images:
                    break
            except Exception:
                continue
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            time.sleep(scroll_pause * 1.5)
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break
        last_h = new_h
    return imgs_found[:max_images]

# -----------------------------
# Download candidates, dedupe, quality-check, save metadata
# -----------------------------
def run_scrape(query, max_per_site=24, driver_headless=True):
    driver = create_driver(headless=driver_headless)
    try:
        all_candidates = []
        print(f"[seed] unsplash -> searching for: {query}")
        try:
            unsplash_cands = scrape_unsplash_search(driver, query, max_images=max_per_site)
            print(f"[scrape] found {len(unsplash_cands)} candidates on unsplash")
            all_candidates.extend(unsplash_cands)
        except Exception as e:
            print("[error] Unsplash scrape failed:", e)

        print(f"[seed] pexels -> searching for: {query}")
        try:
            pexels_cands = scrape_pexels_search(driver, query, max_images=max_per_site)
            print(f"[scrape] found {len(pexels_cands)} candidates on pexels")
            all_candidates.extend(pexels_cands)
        except Exception as e:
            print("[error] Pexels scrape failed:", e)

        print(f"[pipeline] total raw candidates: {len(all_candidates)}")

        # download + dedupe + quality
        metadata = []
        seen_hashes = set()
        idx = 0
        for cand in all_candidates:
            url = cand.get("img_url")
            if not url:
                continue
            if url.startswith("//"):
                url = "https:" + url
            try:
                # Try HEAD first to filter obviously tiny or non-image responses
                try:
                    head = requests.head(url, headers={"User-Agent": USER_AGENT}, timeout=8, allow_redirects=True)
                    ct = head.headers.get("content-type", "")
                    cl = head.headers.get("content-length")
                    if ct and not ct.startswith("image"):
                        print("[skip] not an image (HEAD):", url)
                        continue
                    if cl and int(cl) < 5_000:
                        print("[skip] tiny content-length (HEAD):", url)
                        continue
                except Exception:
                    pass

                content = download_bytes(url)
                h = sha256_bytes(content)
                if h in seen_hashes:
                    print("[dedupe] duplicate:", url)
                    continue
                w, h_px = inspect_image_bytes(content)
                if w * h_px < MIN_PIXELS:
                    print(f"[quality] skipping small image {w}x{h_px} -> {url}")
                    continue

                # save file
                idx += 1
                ext = ".jpg"
                # try detect format via PIL if needed
                try:
                    im = Image.open(BytesIO(content))
                    fmt = im.format.lower() if im.format else "jpg"
                    if fmt == "png":
                        ext = ".png"
                    elif fmt == "gif":
                        ext = ".gif"
                    elif fmt == "webp":
                        ext = ".webp"
                except Exception:
                    pass

                filename = f"{cand['source']}_{idx}{ext}"
                filepath = os.path.join(IMAGES_DIR, filename)
                with open(filepath, "wb") as fh:
                    fh.write(content)

                seen_hashes.add(sha256_bytes(content))
                meta = {
                    "id": idx,
                    "filename": filepath,
                    "source": cand.get("source"),
                    "img_url": url,
                    "page_url": cand.get("page_url"),
                    "alt": cand.get("alt"),
                    "photographer": cand.get("photographer"),
                    "width": w,
                    "height": h_px,
                    "sha256": sha256_bytes(content)
                }
                metadata.append(meta)
                print(f"[saved] {filepath} ({w}x{h_px})")

                # polite pause
                time.sleep(0.25)
            except Exception as e:
                print("[download_error]", e)
                continue

        # write metadata
        with open(METADATA_PATH, "w", encoding="utf-8") as fh:
            json.dump(metadata, fh, indent=2, ensure_ascii=False)
        print(f"[done] saved {len(metadata)} assets and metadata -> {METADATA_PATH}")
        return metadata
    finally:
        driver.quit()

# -----------------------------
# Optional: display a grid of thumbnails
# -----------------------------
def display_images_from_metadata(metadata, cols=4, thumb_size=(360, 240)):
    files = [m["filename"] for m in metadata if os.path.exists(m["filename"])]
    if not files:
        print("[display] no images to show")
        return
    rows = (len(files) + cols - 1) // cols
    plt.figure(figsize=(cols * 3.5, rows * 2.8))
    for i, f in enumerate(files, 1):
        try:
            im = Image.open(f).convert("RGB")
            thumb = Image.new("RGB", thumb_size, (255,255,255))
            # fit
            im.thumbnail(thumb_size, Image.LANCZOS)
            # paste centered
            x = (thumb_size[0] - im.width) // 2
            y = (thumb_size[1] - im.height) // 2
            thumb.paste(im, (x, y))
            plt.subplot(rows, cols, i)
            plt.imshow(thumb)
            plt.axis("off")
            plt.title(os.path.basename(f), fontsize=8)
        except Exception:
            continue
    plt.tight_layout()
    plt.show()

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    query = input("Enter search query (e.g. 'girl dancing'): ").strip() or "girl dancing"
    metadata = run_scrape(query, max_per_site=24, driver_headless=HEADLESS)
    # display first results
    try:
        display_images_from_metadata(metadata, cols=4, thumb_size=(360,240))
    except Exception as e:
        print("Display failed:", e)


In [1]:
# fetch_from_semantic_map.py
# Requirements:
# pip install selenium webdriver-manager requests pillow openai python-dotenv
# Set OPENAI_API_KEY in env before running

import os
import time
import json
import hashlib
import requests
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv

# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# OpenAI client (new SDK)
from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("WARNING: OPENAI_API_KEY not found in environment. Query expansion via LLM will fail.")

client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

# -----------------------------
# Config (tweak as needed)
# -----------------------------
DEST_DIR = "scrape_assets"
IMAGES_DIR = os.path.join(DEST_DIR, "images")
os.makedirs(IMAGES_DIR, exist_ok=True)
METADATA_PATH = os.path.join(DEST_DIR, "metadata.json")
SEMANTIC_MAP_PATH = "semantic_map.json"

USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
MIN_PIXELS = 120 * 120
HEADLESS = True
MAX_QUERIES = 6        # how many queries LLM should return (up to)
MAX_IMAGES_PER_SITE = 12

# -----------------------------
# Reuse the working scraper functions (same pattern you had)
# -----------------------------
def create_driver(headless=HEADLESS):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument(f"user-agent={USER_AGENT}")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    driver.set_page_load_timeout(60)
    return driver

def choose_best_image_src(src, srcset):
    if srcset:
        try:
            parts = [p.strip() for p in srcset.split(",") if p.strip()]
            return parts[-1].split(" ")[0]
        except Exception:
            pass
    return src

def download_bytes(url, timeout=20):
    headers = {"User-Agent": USER_AGENT}
    r = requests.get(url, headers=headers, timeout=timeout, stream=True)
    r.raise_for_status()
    return r.content, r.headers.get("content-type", "")

def inspect_image_bytes(content):
    try:
        im = Image.open(BytesIO(content))
        return im.size  # (w, h)
    except Exception:
        return (0, 0)

def sha256_bytes(b):
    import hashlib
    return hashlib.sha256(b).hexdigest()

# Unsplash search scraping (Selenium)
def scrape_unsplash_search(driver, query, max_images=MAX_IMAGES_PER_SITE, scroll_pause=1.2):
    url = f"https://unsplash.com/s/photos/{query.replace(' ', '-')}"
    driver.get(url)
    time.sleep(2)
    image_list = []
    seen = set()
    last_h = driver.execute_script("return document.body.scrollHeight")
    while len(image_list) < max_images:
        imgs = driver.find_elements(By.TAG_NAME, "img")
        for img in imgs:
            try:
                src = img.get_attribute("src") or ""
                srcset = img.get_attribute("srcset") or ""
                alt = img.get_attribute("alt") or ""
                best = choose_best_image_src(src, srcset)
                if not best:
                    continue
                # prefer Unsplash images
                if "images.unsplash.com" not in best and "/photos/" not in best:
                    continue
                key = best.split("?")[0]
                if key in seen:
                    continue
                # try to grab page url if possible
                page_url = ""
                try:
                    parent = img.find_element(By.XPATH, "./ancestor::a[contains(@href,'/photos')][1]")
                    page_url = parent.get_attribute("href") or ""
                except Exception:
                    pass
                seen.add(key)
                image_list.append({
                    "source": "unsplash",
                    "img_url": best,
                    "alt": alt,
                    "page_url": page_url
                })
                if len(image_list) >= max_images:
                    break
            except Exception:
                continue
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            time.sleep(scroll_pause * 1.5)
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break
        last_h = new_h
    return image_list[:max_images]

# Pexels search scraping (Selenium)
def scrape_pexels_search(driver, query, max_images=MAX_IMAGES_PER_SITE, scroll_pause=1.2):
    url = f"https://www.pexels.com/search/{query.replace(' ', '%20')}/"
    driver.get(url)
    time.sleep(2)
    image_list = []
    seen = set()
    last_h = driver.execute_script("return document.body.scrollHeight")
    while len(image_list) < max_images:
        imgs = driver.find_elements(By.TAG_NAME, "img")
        for img in imgs:
            try:
                src = img.get_attribute("src") or img.get_attribute("data-src") or ""
                srcset = img.get_attribute("srcset") or ""
                alt = img.get_attribute("alt") or ""
                best = choose_best_image_src(src, srcset) or src
                if not best:
                    continue
                # prefer Pexels-hosted images
                if "images.pexels.com" not in best and "pexels" not in best:
                    continue
                key = best.split("?")[0]
                if key in seen:
                    continue
                page_url = ""
                try:
                    parent = img.find_element(By.XPATH, "./ancestor::a[contains(@href,'/photo/')][1]")
                    page_url = parent.get_attribute("href") or ""
                except Exception:
                    pass
                seen.add(key)
                image_list.append({
                    "source": "pexels",
                    "img_url": best,
                    "alt": alt,
                    "page_url": page_url
                })
                if len(image_list) >= max_images:
                    break
            except Exception:
                continue
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            time.sleep(scroll_pause * 1.5)
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break
        last_h = new_h
    return image_list[:max_images]

# Combined runner wrapper (calls both)
def run_scrape_for_query(query, driver, max_per_site=MAX_IMAGES_PER_SITE):
    results = []
    try:
        us = scrape_unsplash_search(driver, query, max_images=max_per_site)
    except Exception as e:
        print("[unsplash scrape error]", e)
        us = []
    try:
        px = scrape_pexels_search(driver, query, max_images=max_per_site)
    except Exception as e:
        print("[pexels scrape error]", e)
        px = []
    results.extend(us)
    results.extend(px)
    return results

# -----------------------------
# LLM prompt for query generation
# -----------------------------
def build_query_generation_prompt(semantic_map, max_queries=MAX_QUERIES):
    """
    Construct a robust prompt for the LLM to produce short search queries optimized for photo sites.
    NOTE: literal braces in the example JSON are escaped as {{ and }} so f-string doesn't try to format them.
    """
    prompt = f"""
You are an assistant that converts a user's creative media intent into short, effective search phrases optimized
for visual stock/photo sites such as Unsplash and Pexels.

Input semantic_map (JSON):
{json.dumps(semantic_map, indent=2)}

Task:
1. Read the semantic_map. Your goal is to produce up to {max_queries} concise search queries (each 2–6 words) that are likely to return **real photographic assets** on Unsplash/Pexels.
2. Avoid returning the long user prompt verbatim. Instead, produce short keyword-rich phrases combining:
   - main subject noun (people/animal/object)
   - action or pose (if relevant)
   - color / lighting / mood (optional)
   - orientation or framing tokens like "portrait", "landscape", "closeup", "aerial" (optional)
   - genre tags like "street", "portrait", "landscape", "studio"
3. Do NOT include punctuation, quotes, or sentence fragments — only short search phrases (lowercase preferred).
4. Prioritize realistic photographic cues (e.g., "girl dancing portrait golden hour", "clouds dramatic sunset", "dog jumping pool summer") rather than abstract/surreal descriptors (if the semantic_map indicates high realism_score, prefer concrete queries; if abstractness is high, include queries that may match creative composites like "surreal cloud shape", but still short).
5. Return EXACTLY ONE JSON object with key "queries" whose value is a list of unique phrases.

Example valid output (note the braces are shown literally here):
{{"queries": ["girl dancing portrait golden hour", "woman dancing street candid"]}}

Now produce the JSON object for the provided semantic_map.
"""
    return prompt


def ask_llm_for_queries(semantic_map, model="gpt-4o-mini"):
    """
    Ask the LLM to generate short search queries. Returns a dict with key "queries".
    Falls back to heuristic extraction if OpenAI client is not configured.
    """
    if client is None:
        print("OpenAI client not available — falling back to simple keyword extraction from semantic_map")
        intent = semantic_map.get("intent_extraction", {})
        subj = intent.get("primary_subject", "").split()[:3]
        ctx = intent.get("context_scene", "").split()[:3]
        styles = intent.get("style_adjectives", [])[:2] if isinstance(intent.get("style_adjectives"), list) else []
        q1 = " ".join([t for t in subj + ctx + styles if t]).strip()
        q2 = " ".join([t for t in subj + ["portrait"] if t]).strip()
        qs = [q for q in [q1, q2] if q]
        return {"queries": qs[:MAX_QUERIES]}

    prompt = build_query_generation_prompt(semantic_map, max_queries=MAX_QUERIES)
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You convert semantic intent into short search phrases for photo sites."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=300
        )
        txt = resp.choices[0].message.content.strip()
    except Exception as e:
        print("[LLM] request failed:", e)
        return {"queries": []}

    # Try to parse JSON directly, otherwise try to extract the first {...} block
    try:
        parsed = json.loads(txt)
        if isinstance(parsed, dict) and "queries" in parsed:
            return parsed
    except Exception:
        pass

    # Fallback: extract first JSON object from the text
    import re
    m = re.search(r"(\{(?:[^{}]|(?R))*\})", txt, flags=re.DOTALL)
    if not m:
        # simpler fallback: find a simple {"queries":[...]} pattern
        m = re.search(r"(\{.*\"queries\"\s*:\s*\[.*?\].*\})", txt, flags=re.DOTALL)

    if m:
        try:
            parsed = json.loads(m.group(1))
            if isinstance(parsed, dict) and "queries" in parsed:
                return parsed
        except Exception:
            pass

    # Last resort: try to read lines and extract quoted phrases or lines
    candidates = []
    for line in txt.splitlines():
        line = line.strip().strip('-• ')
        # if line contains a quoted phrase
        qmatch = re.findall(r'"([^"]{2,80})"|\'([^\']{2,80})\'', line)
        if qmatch:
            for a, b in qmatch:
                candidates.append((a or b).strip())
            continue
        # if line is short (2-6 words), accept it
        if 1 < len(line.split()) <= 6:
            # remove punctuation
            cleaned = re.sub(r'[^\w\s-]', '', line).strip()
            if cleaned:
                candidates.append(cleaned)

    # dedupe and trim
    uniq = []
    for c in candidates:
        cl = c.lower()
        if cl not in [u.lower() for u in uniq]:
            uniq.append(c)
    if uniq:
        return {"queries": uniq[:MAX_QUERIES]}

    # If nothing parsed, return empty
    print("[LLM] Could not parse queries from response. Raw response:")
    print(txt)
    return {"queries": []}

# -----------------------------
# Main orchestrator
# -----------------------------
def main():
    # load semantic map created earlier
    if not os.path.exists(SEMANTIC_MAP_PATH):
        raise FileNotFoundError(f"{SEMANTIC_MAP_PATH} not found. Run prompt-understanding/decision stages first.")
    with open(SEMANTIC_MAP_PATH, "r", encoding="utf-8") as fh:
        semantic_map = json.load(fh)

    # ask LLM to create concise queries
    print("[llm] generating photo-search queries from semantic_map...")
    qobj = ask_llm_for_queries(semantic_map)
    queries = qobj.get("queries", [])[:MAX_QUERIES]
    queries = [q.strip() for q in queries if q and len(q.strip()) > 1]
    if not queries:
        print("No queries produced by LLM — aborting.")
        return

    print("[llm] queries:", queries)

    # run scraping for each query
    driver = create_driver(headless=HEADLESS)
    try:
        all_metadata = []
        seen_hashes = set()
        idx = 0
        for q in queries:
            print(f"[pipeline] scraping for query: '{q}'")
            candidates = run_scrape_for_query(q, driver, max_per_site=MAX_IMAGES_PER_SITE)
            print(f"[pipeline] candidates returned: {len(candidates)}")
            for cand in candidates:
                url = cand.get("img_url")
                if not url:
                    continue
                if url.startswith("//"):
                    url = "https:" + url
                # quick HEAD filter
                try:
                    head = requests.head(url, headers={"User-Agent": USER_AGENT}, timeout=8, allow_redirects=True)
                    ctype = head.headers.get("content-type","")
                    cl = head.headers.get("content-length")
                    if ctype and not ctype.startswith("image"):
                        continue
                    if cl and int(cl) < 3000:
                        continue
                except Exception:
                    pass
                # download
                try:
                    content, ctype = download_bytes(url)
                except Exception as e:
                    print("[download failed]", e)
                    continue
                digest = sha256_bytes(content)
                if digest in seen_hashes:
                    print("[dedupe] duplicate image, skipping")
                    continue
                w, h = inspect_image_bytes(content)
                if w * h < MIN_PIXELS:
                    print("[quality] too small, skipping")
                    continue
                idx += 1
                # choose extension from content type or PIL
                ext = ".jpg"
                if "png" in ctype:
                    ext = ".png"
                elif "gif" in ctype:
                    ext = ".gif"
                else:
                    # try PIL format fallback
                    try:
                        im = Image.open(BytesIO(content))
                        fmt = (im.format or "").lower()
                        if fmt == "png":
                            ext = ".png"
                        elif fmt == "webp":
                            ext = ".webp"
                    except Exception:
                        pass
                fname = f"{cand.get('source','img')}_{idx}{ext}"
                fpath = os.path.join(IMAGES_DIR, fname)
                with open(fpath, "wb") as fh:
                    fh.write(content)
                seen_hashes.add(digest)
                meta_item = {
                    "id": idx,
                    "filename": fpath,
                    "source": cand.get("source"),
                    "img_url": url,
                    "page_url": cand.get("page_url"),
                    "alt": cand.get("alt"),
                    "query_used": q,
                    "width": w,
                    "height": h,
                    "sha256": digest
                }
                all_metadata.append(meta_item)
                print(f"[saved] {fpath} ({w}x{h})")
                time.sleep(0.2)
        # write metadata file and append to semantic_map
        with open(METADATA_PATH, "w", encoding="utf-8") as fh:
            json.dump(all_metadata, fh, indent=2, ensure_ascii=False)
        semantic_map["fetched_assets"] = all_metadata
        # Save updated semantic map
        with open(SEMANTIC_MAP_PATH, "w", encoding="utf-8") as fh:
            json.dump(semantic_map, fh, indent=2, ensure_ascii=False)

        print(f"[done] scraped and saved {len(all_metadata)} assets. metadata -> {METADATA_PATH}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


[llm] generating photo-search queries from semantic_map...
[llm] queries: ['cup of coffee rainy morning', 'coffee on window sill', 'rainy window coffee', 'morning coffee cozy', 'coffee cup indoors', 'rainy day coffee']
[pipeline] scraping for query: 'cup of coffee rainy morning'
[pipeline] candidates returned: 24
[quality] too small, skipping
[saved] scrape_assets\images\unsplash_1.jpg (832x832)
[saved] scrape_assets\images\unsplash_2.jpg (2000x3000)
[quality] too small, skipping
[saved] scrape_assets\images\unsplash_3.jpg (2000x1333)
[quality] too small, skipping
[saved] scrape_assets\images\unsplash_4.jpg (2000x3000)
[quality] too small, skipping
[quality] too small, skipping
[saved] scrape_assets\images\unsplash_5.jpg (2000x3000)
[saved] scrape_assets\images\pexels_6.jpg (1376x768)
[saved] scrape_assets\images\pexels_7.jpg (1376x768)
[saved] scrape_assets\images\pexels_8.jpg (1376x768)
[saved] scrape_assets\images\pexels_9.jpg (1376x768)
[saved] scrape_assets\images\pexels_10.jpg (1